# Model Interpretability

## Importance of Interpretability

Making models interpretable for data scientists and business stakeholders helps with:
- Model debugging - Why did my model make this mistake?
- Feature Engineering - How can I improve my model further?
- Detecting ethical issues - Does my model discriminate based on some feature?
- Human-AI cooperation - How can I understand and trust the model’s decisions?
- Regulatory compliance - Does my model satisfy legal requirements?

Classfication of model based on interpretablility:
- **Glass box model:** models that are interpretable due to their structure, such as linear regression and imple decision tree model.
- **Black box model:** models with a lot of parameters or complex interactions between features; hard to explain relationship between features and target and the relative importance of features in making prediction.

Sometimes we might face implicit constraints on the interpretability of the model, due to tech stack (e.g. production system can only deploy parametrized models) or user needs (e.g. model must be a rule-based system that is editiable)

## Taxonomy of Interpretation Methods

### Classifications of Interpretation Methods:
- **Intrinsic interpretability vs post hoc:** We distinguish interpretation of models which are intrinsically interpretable due to simple structures (e.g. linear regression) and of such that lacked intrinsic interpretablability (e.g. GBM).

- **Model-specific vs. model-agnostic:** Model-specific tools are limited to a class of models, whereas model-agnostic methods can be applied to any model produding predictions based on features.

- **Local vs. global:** Global methods explain the general behaviour of the model (usually explain the change in conditional expectation of the target variable w.r.t to change in feature), while local methods decompose individual predictions.

### Python Libraries for Interpretability:
- DALEX: a package for model agnostic exploration and interpretation.
- InterpretML: an open-source package that incorporates state-of-the-art machine learning interpretability.
- Scikit-learn: it also offers some basic interpretability tools (e.g. PDPs).
- SHAP: a game theoretic approach to explain the output of any machine learning model (a method avaliable in DALEX).

## Global Model Agnostic Methods

Global methods describe the average behavior of a machine learning model. They are often expressed as expected values based on the distribution of the data. We will cover:
- **Partial dependence plots:** Illustrates the average feature effect marginalizing over other features.
- **Cumulated local effect plots (ALE):** Corrects feature effect for feature dependence.
- **Feature interaction (H2-statistic):** Quantifies to what extent the prediction is driven by joint effects of features.

### Partial Dependence Plots
* Partial dependence plots show the marginal effect of one (univariate PDP) or two (bivariate PDP) features on the prediction.
* we usually do not plot partial dependence w.r.t more than 2 features, but we can compute them (it is just not easy to visulize it)
* The univariate partial dependece function for feature s is:
$$\hat{f}_s(x_s) = \mathbb{E}_{X_C}[\hat{f}(x_s, X_c)] = \int \hat{f}(x_s, X_c)\, dP(X_c)$$
* where:
    * $x_s$: feature for which PDP is computed
    * $X_c$: all other features
    * $\hat{f}(.)$: the estimator (model) we fitted
* Partial dependence works by marginalizing (averaging) the preditction over the distribution of the features in set C, so that the influence from feautures in set C are removed, and the function shows the relationship between the features s and the predicted outcome.

Practically, the partial dependece function is estimated by calculating averages in the training data,
$$\hat{f}_S(\mathbf{x}_S) = \frac{1}{n} \sum_{i=1}^{n} \hat{f}(\mathbf{x}_S, \mathbf{x}_C^{(i)})$$
* where:
    * $\mathbf{x}_C^{(i)}$ are actual feature values for the features in set C of observation i
    * $n$ is the number of observations in the dataset

To generate the plot of partial dependece:
1. Select feature $x_s$
2. Define grid for feature (numeric or categorical) on domain of $x_s$
3. Iterate over grid:
    * Set grid value for feature $x_s$
    * Compute and average predictions based on the above formula
4. Plot average predictions over grid

Generate partial dependece plot using Dalex:

In [3]:
import dalex as dx
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

titanic = dx.datasets.load_titanic()
X = titanic.drop(columns='survived')
y = titanic.survived

preprocess = make_column_transformer(
    (StandardScaler(), ['age', 'fare', 'parch', 'sibsp']),
    (OneHotEncoder(), ['gender', 'class', 'embarked'])
)

titanic_rf = make_pipeline(
    preprocess,
    RandomForestClassifier(max_depth=3, n_estimators=500)
)
titanic_rf.fit(X, y)

titanic_rf_exp = dx.Explainer(titanic_rf, X, y,
                              label="Titanic RF Pipeline", verbose=0)

pd_rf = titanic_rf_exp.model_profile(variables=['age', 'fare'])
pd_rf.plot()


Calculating ceteris paribus: 100%|██████████| 2/2 [00:00<00:00, 13.90it/s]


- The PDP function estimated based on the train set can be treated as an average over all individual conditional expectation functions (ICE function for an observation simply describe how prediction change when the feature s changes, keeping all the other features constant at the observed level).
- Hence, we can also partial dependece function by some categorical variable (compute average of ICEs by group to derive PDPs by group).
- For example, we can group the partial dependece function of age by gender, to explore the effect of interaction between age and gender (e.g. maybe for male, higher age correpsond to lower expected survival rate, while for female is the opposite).

**Partial dependence plots advantage:**
- Simple and intuitive way to summarize feature effect.
- Easyn to explain to stakeholders.

**Partial dependence plots disadvantage:**
- For large datasets it is expensive to compute. Yet, the measure is quite robust to sampling (e.g. not compute PDP based on whole training set, instaed, based on a sample from training set).
- It might be problematic to not show the feature s distribution. Omitting the distribution can be misleading because we might overinterpret regions with almost no observations.
- The assumption of independence is the biggest issue with PD plots; for correlated features they are biased, since they may create unexisted data points in areas of the feature distribution where the actual probability is very low.
- For example, we may use weight and height to predict how fast a person walks; we can create a PDP of feauture height; for the computation of the PDP at a certain height (200 cm), we average over the sample distribution of weight, which might include a weight below 50 kg, which is unrealistic for a 2 meter person. Hence, the PDP is biased.

### Accumulated Local Effect

Marginal plot as a solution of feature correlation?
- The drawback of PDPs is the potential correlation between features.
- Instead of computing the averge of predictions with given $x_s$ over unconditional distribution of $X_c$, we could compute the average predictions over conditional distribution of $X_c$; this is known as Marginal Plots.
- Marginal plot: $\hat{f}_{S,M}(x_s)
= \mathbb{E}_{X_C}\!\left[\,\hat{f}_S(x_s, X_C)\mid X_S = x_s\,\right]
= \int \hat{f}(x_s)\, dP(X_C \mid X_S = x_s)$

- However, marginal plot measure the combined effect of the features due to the correlation.
- Consider the previous example of running speed; suppose weight and height is highly positively correlated.
- If we compute the marginal plot at height=200, we got the predictions based on height=200 averaged over the conditional distribution of weight given height=200.
- With the positive correlation, we expect weights for these averaged predictions are also very high.
- Then, if the marginal plot at height=200 shows a very low running speed, we cannot tell whether it is the effect of height=200 or due to the correlated heavy weight.

**Accumulated Local Effect (ALE) Plot:**
- ALE plot is based on the similar idea as marginal plot, but disentangle the feature effects.
- ALE plots solve this problem by calculating – also based on the conditional distribution of the features $X_c$ given $x_s$ – expected difference in predictions with given $x_s$ instead of averages of predictions. 
- For the effect of height on running speed at height=200, the ALE method uses all observations with height within 200 +- k (k dependes on window size we set)
- It gets the model predictions pretending these observations have height 200+k, minus the prediction pretending they were 200-k; the values of other features are at the original observed level.
- The differences are then averaged to get the local effect of $x_s$ on running speed at height 200; we then sum the local effects from height=0 to height=200 to get the ALE (that why called accumulated local effect) of height on running speed.
- This gives us the pure effect of the height and does not mix the effect with the effects of correlated weights. The use of differences blocks the effect of other features. 

**ALE Method Detail:**

* Formula:
$$
\hat{f}_{S,\mathrm{ALE}}(x_S)
= \int_{z_{0,S}}^{x_S} 
\mathbb{E}_{X_C \mid X_S = z_S}\!\left[\,\hat{f}^{\,S}(X_S, X_C) \mid X_S = z_S\,\right] dz_S 
- \text{constant}
$$
$$
= \int_{z_{0,S}}^{x_S} 
\left( \int_{x_C} \hat{f}^{\,S}(z_S, X_C)\, dP(X_C \mid X_S = z_S) \right) dz_S 
- \text{constant}
$$

* We minus a constant at the end to center the ALE plot so that the average effect over the data is zero.

* Estimation based on trainning set:
$$
\hat{\tilde{f}}_{j,\mathrm{ALE}}(\mathbf{x_j})
= \sum_{k=1}^{k_j(\mathbf{x_j})}
\frac{1}{n_j(k)}
\sum_{i : x_j^{(i)} \in N_j(k)}
\left[
\hat{f}(z_{k,j}, \mathbf{x}^{(i)}_{-j})
-
\hat{f}(z_{k-1,j}, \mathbf{x}^{(i)}_{-j})
\right]
$$
$$
\hat{f}_{j,\mathrm{ALE}}(\mathbf{x})
= \hat{\tilde{f}}_{j,\mathrm{ALE}}(\mathbf{x})
- \frac{1}{n} \sum_{i=1}^{n} \hat{\tilde{f}}_{j,\mathrm{ALE}}(x_j^{(i)})
$$

* where:
    * we divide the range of $x_j$ (the feature $x_s$) in to grids with value $z$
    * the intervals in the grid is denoted $k$
    * $k_j(\mathbf{x_j})$ refers to the interval that include value $x_j$
    * $N_j(k)$ refers to the neighborhood k, which include all observations with $x_s$ within the interval k
    * $z_{k,j}$ denotes the upper bound in interval k, $z_{k-1,j}$ denotes the lower bound

* we again centered the effect so that the mean effect is zero.
* the value of the ALE can be interpreted as the main effect of the feature j at a certain value compared to the average prediction of the data. 
* for example, an ALE estimate of -2 at $x_j=3$ means that when the j-th feature has value 3, then the prediction is lower by 2 compared to the average prediction.

**Advantages of ALE:**
- Unlike partial dependence plots (PDP), ALE plots are not biased by correlations between features.
- Generally faster to compute compared to other model interpretation techniques, especially for large datasets.

**Disadvantages of ALE:**
- Interpretation is more complex (to explain to stakeholder) due to conditional difference computation.
- Sensitivity to the specific binning (the num of intervals in the grid of $x_j$).

### H2-Statistic
- It is a way to measure the interaction strength between two features
- The idea is to measure how much variation of the prediction depends on the interaction of the two features. 

**Principle behind H2-statistic:**
- If two features do not interact, we can decompose the PDP simply by summing the univariate PDPs (assuming the partial dependence functions are centered at zero by subtracting the averge level of PD): $PD_{jk}(x_j, x_k) = PD_j(x_j) + PD_k(x_k)$ 
- We want a test statistic which is 0, if there is no interaction, i.e. the bivariate PD is simply the sum of the univariate PDs, and which is 1, if the variation of the prediction is purely coming from the interaction.
- We can then define H2-statistics between two features as: 
$$
H_{jk}^2
= \dfrac{
\sum_{i=1}^{n} \left( PD_{jk}(x_j^{(i)}, x_k^{(i)}) - PD_j(x_j^{(i)}) - PD_k(x_k^{(i)}) \right)^2
}{
\sum_{i=1}^{n} PD_{jk}^2(x_j^{(i)}, x_k^{(i)})
}
$$
- where n denotes the number of observations in the training set
- We can also define H2-statistics as below to test for strength of interaction between $x_j$ and all other features:
$$
H_j^2
= \dfrac{
\sum_{i=1}^{n} \left[\, \hat{f}(\mathbf{x}^{(i)}) - PD_j(x_j^{(i)}) - PD_{-j}(\mathbf{x}_{-j}^{(i)}) \,\right]^2
}{
\sum_{i=1}^{n} \left( \hat{f}(\mathbf{x}^{(i)}) \right)^2
}
$$
- note that $\hat{f}(\mathbf{x}^{(i)})$ is equivalent to the PD of all features.
- python package for implmenting H2: https://selfexplainml.github.io/PiML-Toolbox/_build/html/index.html

**Advantages of H2:**
- It has meaningful interpretation: share of variance in partial dependence output explained by interaction.
- Scale free and dimensionless, so it can be compared across features and models.
- We can estimate a GBM with all features, compute H2 for all feature interaction, find the strongest feature interaction, include them in a parametric model (e.g. GLM)

**Disadvantages of H2:**
- Usually computationally expensive.

## Local model-agnostic methods

So far we have considered methods which are explaining the overall model behaviour. Local methods explain individual predictions. What we will cover:
- Individual conditional explanation curves (ICE)
- Shapley values

### Individual Conditional Expectation
- They are the building block for PDPs, illustrating how predictions for specific observations change, if only one feature is changed. 
- They have the same drawbacks as for PDPs, when features are correlated, some predictions in the lines might be invalid, because correpsonding observation may not even exist.
- The advantage relative to PDPs is, is that we get a better view on the heterogeneity of predictions.
- Formula: $\hat{f}_s(x_s)^i = \hat{f}(x_s, X_c^i)$

### Shapley Values

We often face the scenario that we have a high-dimensional model and the question is how much does each feature contribute to the specific prediction. To answer this question, we want to decompose the prediction into additive contributions of features.
* Shapley value is a method to conduct additive decomposition; the theory originates from cooperative game theory. 
* Features can be interpreted as players and predictions are the payout. 
* The problem to solve is: How to distribute the model’s prediction across features.
* The Shapley value solve the problem with following properties:
    * Efficiency: The feature contributions must add up to the difference of the **specific prediction** and the **average**:
    $$ \sum_i \phi_i = \hat{f}(x) - \mathbb{E}[\hat{f}(X)]$$
    * Symmetry: If features contribute equally, then $ϕ_i = ϕ_j$
    * Dummy: A feature that does not change the prediction should have Shapley value of $ϕj = 0$.
    * Additivity: If predictions are additive, so are Shapley values (e.g. for a model that sum result from multiple trees to form preditction, the Shapley values of the final model should also be the sum of Shapley values of individual trees).
* In a simple linear regression, the Shapley value of feature x_s for prediction i is simply $PD(x_s^i) - \mathbb{E}[PD(x_s)]$

![my plot](graphs/shapley_values.png)

When plotting Shapley values for multiple observations, we get a so-called **beeswarm plot**:
- This indicates which features have the largest impact on the predictions for given feature values.
- The feature with more dispersed Shapley values have the greater impact on the predictions.
- higher dispersion mean that, for more preditcions, the feature has either more negative or positive contribution.

![my plot](graphs/beeswarm.png)

Exact formula for Shapley values:
$$
\phi_i
= \sum_{S \subseteq F \setminus \{i\}}
\frac{|S|! \, (|F| - |S| - 1)!}{|F|!}
\left[
f_{S \cup \{i\}}(x_{S \cup \{i\}}) - f_S(x_S)
\right]
$$

where:
- $\phi_i $: Shapley value of feature i
- $ F $: set of all features despite i
- $ S $: any subset of F (iterate through all possible subset of F)
- $ \frac{|S|! \, (|F| - |S| - 1)!}{|F|!} $:  subset combinations
- $ f_{S \cup \{i\}} $: prediction made by model using features in S plus feature i (marginalizing left features by averaging predictions over the joint distribution of left features)
- $ f_S $: prediction made by model using features only in S (without feature i)


### SHAP
- The Shapley value requires a lot of computing time (and usually impossible), as we would have to compute the prediction marginalized over some features for all possible combinations of subsets of features
- SHAP is a good approximation for Shapley values purposed by Lundberg & Lee (2017).
- More details: Interpretable ML Chapter 9.